In [1]:
#importing all the requried modules
import numpy as np 
import pandas as pd 
import pymysql
from sqlalchemy import create_engine
from sklearn.metrics import confusion_matrix 
from sklearn.model_selection import train_test_split 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from pandas import ExcelWriter
from pandas import ExcelFile
from sklearn import preprocessing
%matplotlib notebook
import matplotlib.pyplot as plt, mpld3
from matplotlib import rcParams
from matplotlib.cm import rainbow
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import State, Input, Output
import dash_bootstrap_components as dbc
import dash_auth
from flask import Flask, redirect
from pathlib import Path
import dash_table
import plotly.graph_objs as go
#connecting to a database
engine = create_engine('mysql+pymysql://root:@localhost/student')
mycursor= engine.connect()



#creating model
def importdata(): 
    balance_data = pd.read_excel('dataset.xlsx') 
    #data_tests=pd.read_excel('testdata.xlsx')
    #original=data_tests
    
    #balance_data = balance_data.apply(le.fit_transform)
    #data_tests = data_tests.apply(le.fit_transform)
    return balance_data
# Function to split the dataset 
def splitdataset(balance_data): 

    # Separating the target variable 
    X = balance_data.values[:, 2:4] 
    Y = balance_data.values[:, 4]
    #print(X)
    # Splitting the dataset into train and test 
    X_train, X_test, y_train, y_test = train_test_split(  X, Y, test_size = 0.3, random_state = 100) 
    return X, Y, X_train, X_test, y_train, y_test 

# Function to perform training with giniIndex. 
def train_using_gini(X_train, X_test, y_train,y_test): 
    
    # Creating the classifier object 
    clf_gini = DecisionTreeClassifier(criterion = "gini", random_state = 100, min_samples_leaf=2) 
    # Performing training 
    clf_gini.fit(X_train, y_train) 
    return clf_gini 
# Function to perform training with entropy. 
def tarin_using_entropy(X_train, X_test, y_train): 

    # Decision tree with entropy 
    clf_entropy = DecisionTreeClassifier( criterion = "entropy", random_state = 100, max_depth = 3, min_samples_leaf = 2) 
    # Performing training 
    clf_entropy.fit(X_train, y_train) 
    return clf_entropy 
def predictionres(data_tests, clf_object): 
    originaldata=data_tests
    # Predicton on test with giniIndex 
    datapred=[]
    #print(data_tests)
    #print(clf_object)
    datay_pred = clf_object.predict(data_tests) 
    #print("Predicted values:") 
    #print(datay_pred)
    datapred=list( datay_pred)
    #print(datapred)
    return datapred

#def main(): 
# Building Phase 
#le = preprocessing.LabelEncoder()
data= importdata() 
X, Y, X_train, X_test, y_train, y_test = splitdataset(data) 
clf_gini = train_using_gini(X_train, X_test, y_train,y_test) 
clf_entropy = tarin_using_entropy(X_train, X_test, y_train) 
#creating apps for frontend

app = dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])
app2= dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])
app3= dash.Dash(external_stylesheets=[dbc.themes.CERULEAN])

#app4= dash.Dash()
#app5=dash.Dash()
app.config['suppress_callback_exceptions']=True
app2.config['suppress_callback_exceptions']=True
app3.config['suppress_callback_exceptions']=True
#app4.config['suppress_callback_exceptions']=True
#app5.config['suppress_callback_exceptions']=True
#implimentation of app layouts
app.layout = html.Div(style={'textAlign': 'center','color': 'black'}
,children=[
        html.H1('STUDENT PERFORMANCE ANALYSIS SYSTEM USING MACHINE LEARNING',style={'text':'bold','textAlign': 'center','color':'red'}),
        html.Br(),html.Br(),
        dbc.Label("Login as "),
        html.Div(dbc.RadioItems(id='radio',options=[{'label': 'Student', 'value': 's'},{'label': 'Faculty', 'value': 't'}]),
        style={'textAlign': 'center','color':'black'}), 
        html.Br(),
        html.Div( dbc.Button('Submit', id='button',color="primary", className="mr-1"),style={'textAlign': 'center','color':'gray'}),
        html.Br(),html.Br(),
        dcc.ConfirmDialog(id='confirm',message='Invalid User try With Proper Username/Password '),
        html.Div(id='page-1-content',style={'textAlign': 'center','backgroundColor': 'white'}),
])
'''app.css.append_css({
    "external_url": "https://codepen.io/chriddyp/pen/bWLwgP.css"
})'''
app2.layout=html.Div([html.Div(id='page-2-content',style={'textAlign': 'center','color': 'black'})])
app3.layout=html.Div([html.Div(id='page-3-content',style={'textAlign': 'center','color': 'black'})])
#app4.layout=html.Div([html.Div(id='page-4-content',style={'textAlign': 'center','backgroundColor':'white'})])
#app5.layout=html.Div([html.Div(id='page-5-content',style={'textAlign': 'center','backgroundColor':'white'})])

#app5.layout = html.Div([generate_table(df)])
#implimenting functionality of buttons
@app.callback(dash.dependencies.Output('page-1-content', 'children'),
              [dash.dependencies.Input('button', 'n_clicks')],
                state=[State('radio', 'value')])
def compute(n_clicks,radio):
    if(radio=='t'):
        return dbc.Label("Username  :",className="mr-3"),dcc.Input(type="text", id="inp5", placeholder="Enter email"),html.Br(),html.Br(),dbc.Label("Password  :",className="mr-3"),dcc.Input(id='inp6',type='password',placeholder="Password"),html.Br(),html.Br(),html.Div([html.A('Forget-Password', href='http://127.0.0.1:2018/', target='_self'),html.Br(),html.Br(),dbc.Button('Login', id='button1',color="primary", className="mr-1",n_clicks=0)]),html.Br(),app2.layout
    if(radio=='s'):
        return dbc.Label('Rollnumber : ',className="mr-3"),dcc.Input(id='inp1',type='text',placeholder="Enter Roll No"),dbc.Label('    Subject : ',className="mr-3"),dcc.Input(id='inp2',type='text',placeholder="Enter Subject"),dbc.Label('    Mid1 Marks : ',className="mr-3"),dcc.Input(id='inp3',type='text',placeholder="Enter Mid1 Marks"),dbc.Label('    Mid2 Marks : ',className="mr-3"),dcc.Input(id='inp4',type='text',placeholder="Enter Mid2 Marks"),html.Br(),html.Br(),dbc.Button('Predict', id='button2',color="primary", className="mr-1",n_clicks=0),html.Br(),html.Br(),app3.layout


@app.callback(dash.dependencies.Output('page-2-content', 'children'),
              [dash.dependencies.Input('button1', 'n_clicks')],
                state=[State('radio', 'value'),State('inp5', 'value'),State('inp6','value')])
def compute2(n_clicks,radio,inp5,inp6):
    if(int(n_clicks)>0):
        #print("hi")
        klis=[]
        p=mycursor.execute("select password from passwords where email='{}'".format(inp5))
        for i in p:
            #print(i)
            for j in i:
                #print(j)
                klis.append(j)
                #print(klis)
        #print(klis)
        if(radio=='t' and (inp6 in klis)):
            #print("hi")
            return html.Label('valid user'),html.Br(),html.Div([html.A('upload excel sheet', href='http://127.0.0.1:2016/', target='_self')]),html.Br(),html.Div([html.A('predict from database', href='http://127.0.0.1:2017/', target='_self')])

@app.callback(dash.dependencies.Output('confirm', 'displayed'),
            [dash.dependencies.Input('button1', 'n_clicks')],
            state=[State('radio', 'value'),State('inp5', 'value'),State('inp6','value')])
def popup(n_clicks,radio,inp5,inp6):
    if(int(n_clicks)>0):
        klis1=[]
        engine1 = create_engine('mysql+pymysql://root:@localhost/student')
        mycursor1= engine1.connect()
        p1=mycursor1.execute("select password from passwords where email='{}'".format(inp5))
        for i in p1:
            #print(i)
            for j in i:
                #print(j)
                klis1.append(j)
        #print(klis1)
        if(inp6 not in klis1):
                    return True
        
@app.callback(dash.dependencies.Output('page-3-content', 'children'),
              [dash.dependencies.Input('button2', 'n_clicks')],
                state=[State('radio', 'value'),State('inp1', 'value'),State('inp2', 'value'),State('inp3', 'value'),State('inp4', 'value')])
def compute3(n_clicks,radio,inp1,inp2,inp3,inp4):
    if(int(n_clicks)>0):
        if(radio=='s'):
            df=pd.DataFrame({ 
                    "Rollnumber":[inp1],
                   "subject":[inp2], 
                   "mid1":[inp3],
                     "mid2":[inp4]}) 
            #print(df)
            df10=df.values[:, 2:4] 
            df9= pd.DataFrame(df10)
            gres= predictionres(df9,clf_gini)
            eres=predictionres(df9,clf_entropy)
            return 'your Expected Grade  is "{}" '.format(gres[0]),html.Br(),html.Br()


In [ ]:
if __name__ == '__main__':
    app.run_server(port=2020)
    
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:2020/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Apr/2020 00:54:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:28] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:28] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:30] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:54:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:04] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:04] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [03/Apr/2020 00:59:07] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - -